In [7]:
# For optimization using pygmo
import numpy as np

# For Plotting
import matplotlib.pyplot as plt

# For computing acceleration and potential
import polyhedral_gravity as model

# General
from typing import Union






######################################################
#              Runge-Kutta-Fehlberg 78               #
######################################################

def butcher_table_rkf78() -> Union[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """ Butcher table/tablau for Runge-Kutta-Fehlberg 7(8) method.

    Returns:
        All table values defined for the RKF-78 method. 
    """
    a0 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
    a1 = [2/27, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    a2 = [1/36, 1/12, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    a3 = [1/24, 0, 1/8, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    a4 = [5/12, 0, -25/16, 25/16, 0, 0, 0, 0, 0, 0, 0, 0]
    a5 = [1/20, 0, 0, 1/4, 1/5, 0, 0, 0, 0, 0, 0, 0]
    a6 = [-25/108, 0, 0, 125/108, -65/27, 125/54, 0, 0, 0, 0, 0, 0]
    a7 = [31/300, 0, 0, 0, 61/225, -2/9, 13/900, 0, 0, 0, 0, 0]
    a8 = [2, 0, 0, -53/6, 704/45, -107/9, 67/90, 3, 0, 0, 0, 0]
    a9 = [-91/108, 0, 0, 23/108, -976/135, 311/54, -19/60, 17/6, -1/12, 0, 0, 0]
    a10 = [2383/4100, 0, 0, -341/164, 4496/1025, -301/82, 2133/4100, 45/82, 45/164, 18/41, 0, 0]
    a11 = [37/205, 0, 0, 0, 0, -6/41, -3/205, -3/41, 3/41, 6/41, 0, 0]
    a12 = [-1777/4100, 0, 0, -341/164, 4496/1025, -289/82, 2193/4100, 51/82, 33/164, 12/41, 0, 1]

    b = [0, 2/27, 1/9, 1/6, 5/12, 1/2, 5/6, 1/6, 2/3, 1/3, 1, 0, 1]

    c = [41/840, 0, 0, 0, 0, 34/105, 9/35, 9/35, 9/280, 9/280, 41/840, 0, 0]
    c_hat = [0, 0, 0, 0, 0, 34/105, 9/35, 9/35, 9/280, 9/280, 0, 41/840, 41/840]

    a = np.array([a0, a1, a2, a3, a4, a5, a6, a7, a8, a9, a10, a11, a12])
    b = np.array(b)
    c = np.array(c)
    c_hat = np.array(c_hat)

    return b, a, c, c_hat
    


# Used by all RK-type algorithms
def compute_motion(x: np.ndarray, t: float) -> np.ndarray:
    """ State update equation for RK-type algorithms. 

    Args:
        t : Time value corresponding to current state
        x : State vector containing position and velocity expressed in three dimensions.

    Returns:
        State vector used for computing state at the following time step.
    """

    kx = (3 - 4*x)/(2*t)
    
    return np.array(kx)




def new_rkf78(x: np.ndarray) -> np.ndarray:
    """
    Integrates Newton's equations of motion within a user defined time interval
    using Runge-Kutta-Fehlberg 7(8) method.

    Returns:
        Complete trajectory information stored in trajectory_info.
    """

    b, a, c, c_hat = butcher_table_rkf78()


    t0 = 2
    tf = 10
    h0 = 0.01
    n_max = 10000
    tol_min = 1e-12
    tol_max = 1e-12
    h_min = 1e-6
    h_max = 10000

    h = h0
    t = t0
    k = 0

    trajectory_info = np.empty((2, n_max), dtype=np.float64)
    trajectory_info[0:1, k] = x
    trajectory_info[1, k] = t

    while (k < n_max) and (t < tf):
        
        if h < h_min:
            h = h_min
        elif h > h_max:
            h = h_max
        
        k0 = compute_motion(x, t)
        k1 = compute_motion(x + h*(a[1,0]*k0), t + h*b[1])
        k2 = compute_motion(x + h*(a[2,0]*k0 + a[2,1]*k1),t+ h*b[2])
        k3 = compute_motion(x + h*(a[3,0]*k0 + a[3,1]*k1 + a[3,2]*k2), t + h*b[3])
        k4 = compute_motion(x + h*(a[4,0]*k0 + a[4,1]*k1 + a[4,2]*k2 + a[4,3]*k3), t+  h*b[4])
        k5 = compute_motion(x + h*(a[5,0]*k0 + a[5,1]*k1 + a[5,2]*k2 + a[5,3]*k3 + a[5,4]*k4), t + h*b[5])
        k6 = compute_motion(x + h*(a[6,0]*k0 + a[6,1]*k1 + a[6,2]*k2 + a[6,3]*k3 + a[6,4]*k4 + a[6,5]*k5), t + h*b[6])
        k7 = compute_motion(x + h*(a[7,0]*k0 + a[7,1]*k1 + a[7,2]*k2 + a[7,3]*k3 + a[7,4]*k4 + a[7,5]*k5 + a[7,6]*k6), t + h*b[7])
        k8 = compute_motion(x + h*(a[8,0]*k0 + a[8,1]*k1 + a[8,2]*k2 + a[8,3]*k3 + a[8,4]*k4 + a[8,5]*k5 + a[8,6]*k6 + a[8,7]*k7), t + h*b[8])
        k9 = compute_motion(x + h*(a[9,0]*k0 + a[9,1]*k1 + a[9,2]*k2 + a[9,3]*k3 + a[9,4]*k4 + a[9,5]*k5 + a[9,6]*k6 + a[9,7]*k7 + a[9,8]*k8), t + h*b[9])
        k10 = compute_motion(x + h*(a[10,0]*k0 + a[10,1]*k1 + a[10,2]*k2 + a[10,3]*k3 + a[10,4]*k4 + a[10,5]*k5 + a[10,6]*k6 + a[10,7]*k7 + a[10,8]*k8 + a[10,9]*k9), t + h*b[10])
        k11 = compute_motion(x + h*(a[11,0]*k0 + a[11,1]*k1 + a[11,2]*k2 + a[11,3]*k3 + a[11,4]*k4 + a[11,5]*k5 + a[11,6]*k6 + a[11,7]*k7 + a[11,8]*k8 + a[11,9]*k9 + a[11,10]*k10), t + h*b[11])
        k12 = compute_motion(x + h*(a[12,0]*k0 + a[12,1]*k1 + a[12,2]*k2 + a[12,3]*k3 + a[12,4]*k4 + a[12,5]*k5 + a[12,6]*k6 + a[12,7]*k7 + a[12,8]*k8 + a[12,9]*k9 + a[12,10]*k10 + a[12,11]*k11), t + h*b[12])
    
        y = x + h*(c[0]*k0 + c[1]*k1 + c[2]*k2 + c[3]*k3 + c[4]*k4 + c[5]*k5 + c[6]*k6 + c[7]*k7 + c[8]*k8 + c[9]*k9 + c[10]*k10)
        y_hat = x + h*(c_hat[0]*k0 + c_hat[1]*k1 + c_hat[2]*k2 + c_hat[3]*k3 + c_hat[4]*k4 + c_hat[5]*k5 + c_hat[6]*k6 + c_hat[7]*k7 + c_hat[8]*k8 + c_hat[9]*k9 + c_hat[10]*k10 + c_hat[11]*k11 + c_hat[12]*k12)

        trunc_error = np.linalg.norm(y-y_hat)

        if (trunc_error > tol_max) and (h > h_max):
            h = h/2
        
        else:
            k = k + 1
            t = t + h
            x = y_hat

            trajectory_info[0:1, k] = x
            trajectory_info[1, k] = t
            
            if trunc_error < tol_min:
                h = 2*h
    
    trajectory_info = np.array(trajectory_info)
    return trajectory_info[:, 0:k]


x = 5/4
trajectory_info = new_rkf78(x)

print(trajectory_info)
print("Last: ", trajectory_info[:,-1])

[[ 1.25        1.24503765  1.24014881 ...  0.77008285  0.77004266
   0.7700026 ]
 [ 2.          2.01        2.02       ...  9.98        9.99
  10.        ]]
Last:  [ 0.7700026 10.       ]


In [5]:
def f(t,x):
  k = (3 - 4*x)/(2*t)
  return k


def rk45():
  epsilon = 0.00001
  h = 0.01
  t = 2
  w = 5/4
  i = 0

  info_list = np.empty((2,2000), dtype=np.float64)
  info_list[0,0] = w
  info_list[1,0] = t

  while t<10:
    h = min(h, 2-t)
    k1 = h*f(t,w)
    k2 = h*f(t + (h/4), w + (k1/4))
    k3 = h*f(t + (3*h/8), w + (3*k1/32) + (9*k2/32))
    k4 = h*f(t + (12*h/13), w + (1932*k1/2197) - (7200*k2/2197) + (7296*k3/2197))
    k5 = h*f(t + h, w + (439*k1/216) - (8*k2) + (3680*k3/513) - (845*k4/4104))
    k6 = h*f(t + (h/2), w - (8*k1/27) + (2*k2) - (3544*k3/2565) + (1859*k4/4104) - (11*k5/40))
    w1 = w + (25*k1/216) + (1408*k3/2565) + (2197*k4/4104) - (k5/5)
    w2 = w + (16*k1/135) + (6656*k3/12825) + (28561*k4/56430) - (9*k5/50) + (2*k6/55)
    err = np.linalg.norm(w1-w2)/h
    delta = 0.84*(epsilon/err)**(1/4)
    
    if err<=epsilon:
      t = t+h
      w = w1
      i = i+1
      h = delta*h

      info_list[0,i] = w
      info_list[1,i] = t

    else:
      h = delta*h
    

  return info_list

info_list = rk45()

print(info_list)

/var/folders/1q/7vjhndg54wl3ndml8n9wc8m40000gn/T/ipykernel_17661/602976417.py:27: RuntimeWarning: invalid value encountered in double_scalars
  err = np.linalg.norm(w1-w2)/h


KeyboardInterrupt: 

In [6]:
# D-solver
import desolver as de
import desolver.backend as D

# Remember, if we want to use this package we must switch places of t and x in equations of motion it shoulf be: eq_of_motion(t,x) for D.solver to work.
#   as the input to equations_of_motion

# Used by all RK-type algorithms
def compute_motion(t,x):
    """ State update equation for RK-type algorithms. 

    Args:
        t : Time value corresponding to current state
        x : State vector containing position and velocity expressed in three dimensions.

    Returns:
        State vector used for computing state at the following time step.
    """

    kx = (3 - 4*x)/(2*t)
    
    return np.array(kx)




x0=5/4
t0 = 2
tf = 10
h0 = 0.001

D.set_float_fmt('float64')
initial_state = D.array(x0)
a = de.OdeSystem(compute_motion, y0=initial_state, dense_output=True, t=(t0, tf), dt=h0, rtol=1e-12, atol=1e-12)
a.method = "RK87"
a.integrate()
trajectory_info = np.transpose(a.y)
time_info = np.transpose(a.t)


print(trajectory_info)
print(time_info)

Using numpy backend


[1.25       1.24950037 1.246589   1.23350326 1.18515394 1.13950786
 1.09839775 1.06133563 1.02793694 0.99785787 0.97078247 0.94642832
 0.92453827 0.90488051 0.88724393 0.87143672 0.85728502 0.84463063
 0.83332933 0.82325044 0.81427434 0.80629241 0.79920589 0.79292477
 0.78736722 0.78245876 0.77813179 0.77432508 0.77098286 0.77      ]
[ 2.          2.001       2.0068571   2.03383303  2.14384577  2.26598425
  2.39594767  2.53454838  2.68251317  2.84062326  3.00976578  3.19089816
  3.38508563  3.59349146  3.81740659  4.05826101  4.31763033  4.59726075
  4.89909715  5.22528388  5.5782269   5.96060155  6.3753855   6.82591683
  7.31593064  7.84962717  8.431722    9.06750581  9.76298611 10.        ]
